<a href="https://colab.research.google.com/github/dylanesq/2-Months-Project-LLM/blob/main/LlamaIndexV5_HFEvalute_Qwen_Qwen3_1_7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
temp=0.0

In [37]:
pip install llama-index

#Loading

 **SIMPLE DIRECTORY READER**
READING PDFS

In [38]:
from llama_index.core import SimpleDirectoryReader
reader=SimpleDirectoryReader(input_dir="/content/", required_exts=[".pdf"])
document_1=reader.load_data()

# Chunking Randomly
[TokenTextSplitter](https://docs.llamaindex.ai/en/v0.10.17/api/llama_index.core.node_parser.TokenTextSplitter.html#tokentextsplitter)

In [39]:
from llama_index.core.node_parser import TokenTextSplitter
splitter = TokenTextSplitter(
    chunk_size=2048,
    chunk_overlap=200,
    separator=" ",
)
token_nodes = splitter.get_nodes_from_documents(
    document_1, show_progress=True
)

Parsing nodes:   0%|          | 0/172 [00:00<?, ?it/s]

# LLM Setup
*   [HuggingFaceLLM](https://docs.llamaindex.ai/en/v0.9.48/api_reference/llms/huggingface.html#huggingfacellm)
*   [Llama 3.2 3B
](https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct)



#LLM Init.

In [40]:
!pip install llama_index.llms.huggingface
from llama_index.llms.huggingface import HuggingFaceLLM

system prompt

In [41]:
from transformers import AutoTokenizer, AutoModelForCausalLM
model_id = "Qwen/Qwen3-1.7B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,torch_dtype="auto",device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# **Embedding & Vectorization**


Vector Imports

Embedding Imports
* [HuggingFaceEmbeddings](https://docs.llamaindex.ai/en/stable/examples/embeddings/huggingface/)  
*   [LAngchain HF Embedding](https://python.langchain.com/api_reference/huggingface/embeddings/langchain_huggingface.embeddings.huggingface.HuggingFaceEmbeddings.html#huggingfaceembeddings)
*   [SBERT](https://www.sbert.net/)
*   [Llamindex embedding](https://docs.llamaindex.ai/en/stable/api_reference/embeddings/huggingface/)





In [42]:
!pip install llama_index.embeddings.huggingface
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model  = HuggingFaceEmbedding(model_name = "sentence-transformers/all-MiniLM-L6-v2")

In [43]:
system_prompt="""
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
If it is note in the context, say you don't know. Don't try to make up an answer. no extra words.
"""

Wrapper

In [72]:
llm = HuggingFaceLLM(
    #context_window=1024,
    max_new_tokens=512,
    generate_kwargs={"temperature": temp, "do_sample": False},
    system_prompt=system_prompt,
    model=model,
    tokenizer=tokenizer,
    tokenizer_kwargs={"max_length": 256},
)

# Service Context/Settings

Set Up
Edit  : ServiceContext is deprecated.

*   Use llama_index.settings.Settings instead
*   [Migration to Settings Llama Index](https://docs.llamaindex.ai/en/stable/module_guides/supporting_modules/service_context_migration/)


In [73]:
from llama_index.core import Settings
Settings.llm=llm
Settings.embed_model=embed_model
Settings.node_parser=splitter
Settings.chunk_size = 1024

#Indexing

In [74]:
from llama_index.core import VectorStoreIndex
index=VectorStoreIndex.from_documents(document_1)

#Query Engine



*   [Querying LlamaIndex](https://docs.llamaindex.ai/en/stable/module_guides/querying/)
*   [Query Engine Module
](https://docs.llamaindex.ai/en/stable/module_guides/deploying/query_engine/modules/)


Custom Query engine **[working]**

In [75]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

retriever=VectorIndexRetriever(index=index,similarity_top_k=5)
query_engine = RetrieverQueryEngine(retriever=retriever)


**Vanilla Query Response**

In [76]:
prompt="what is working capital? what does it tell you about a company?"

Responses

In [77]:
response=query_engine.query(prompt)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [78]:
from llama_index.core.response.pprint_utils import pprint_response
pprint_response(response,show_source=False)

Final Response: Working capital is the difference between current
assets and current liabilities. It indicates the amount of working
capital available to a company to meet its short-term obligations. If
the working capital is positive, it means the company has sufficient
working capital to meet its short-term obligations. If it is negative,
it means the company has a working capital deficit and may need to
seek a working capital loan from its bankers. ---------------------
Answer the question: what is working capital? what does it tell you
about a company?  The working capital is the difference between
current assets and current liabilities. It indicates the amount of
working capital available to a company to meet its short-term
obligations. If the working capital is positive, it means the company
has sufficient working capital to meet its short-term obligations. If
it is negative, it means the company has a working capital deficit and
may need to seek a working capital loan from its b

In [79]:
norank_retrieved_nodes = retriever.retrieve(prompt)

 # print(f"Retrieved {len(norank_retrieved_nodes)} nodes:")
  #for i, node in enumerate(norank_retrieved_nodes):
   #   print(f"\n--- Node {i + 1} ---")
    #  print(node.get_content())
     # print(f"Score: {node.score}")


! Insert section for Pre rerank evaluation !


# Pre Rerank Evaluation
*Types of Metrics*
See Alammar Pg 257
*   [RAGAS Documentation on the metrics](https://docs.ragas.io/en/stable/concepts/metrics/available_metrics/)
*   [Confident AI Article really GOOOD](https://www.confident-ai.com/blog/lm-evaluation-metrics-everything-you-need-for-llm-evaluation)
*[Hallucination](https://arxiv.org/pdf/2309.05922)

*  [RAGChecker Paper](https://arxiv.org/pdf/2408.08067)

*  [BERTScore](https://arxiv.org/abs/1904.09675)







# Inbuilt Evaluation Faithfulness Relevancy (?)

**GT Answer getting from llama3-8b-8192** via groq API

In [80]:
!pip install groq
from google.colab import userdata
from groq import Groq

client = Groq(api_key=userdata.get('GROQ_API_KEY'))

response_groq = client.chat.completions.create(
    model="llama3-8b-8192",  # You can use other models like llama3-70b-8192
    messages=[
        {"role": "user", "content": prompt}
    ]
)
gt_answer = response_groq.choices[0].message.content.strip()


In [81]:
print("GT Answer :\n\n",gt_answer)

GT Answer :

 Working capital is the amount of liquid assets that a company has available to operate its business, meet its financial obligations, and expand its operations. It is the difference between a company's current assets and its current liabilities. In other words, it is the company's ability to pay its short-term debts and obligations.

Working capital is typically calculated using the following formula:

Working Capital = Current Assets - Current Liabilities

Where:

* Current Assets = Cash, Accounts Receivable, Inventory, Prepaid Expenses, and other assets that are expected to be converted into cash within one year.
* Current Liabilities = Accounts Payable, Accrued Expenses, Taxes Payable, and other liabilities that are due within one year.

Working capital can be further classified into three main components:

1. Cash and Cash Equivalents: This includes cash, checks, and other short-term liquid investments.
2. Accounts Receivable: This is the amount of money that customers

In [82]:
retrieved_context = " ".join([node.text for node in norank_retrieved_nodes])

In [83]:
from llama_index.core.llms import (
    CustomLLM,
    CompletionResponse,
    CompletionResponseGen,
    LLMMetadata,
)
from groq import Groq
from pydantic import PrivateAttr
import os

class GroqLLM(CustomLLM):
    model: str
    _client: Groq = PrivateAttr()

    def __init__(self, model="llama3-70b-8192"):
        super().__init__(model=model)
        self._client = Groq(api_key=userdata.get('GROQ_API_KEY'))

    def complete(self, prompt: str, **kwargs) -> CompletionResponse:
        response = self._client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}]
        )

        return CompletionResponse(text=response.choices[0].message.content.strip())

    def stream_complete(self, prompt: str, **kwargs) -> CompletionResponseGen:
        raise NotImplementedError("Streaming not supported yet")

    @property
    def metadata(self) -> LLMMetadata:
        return LLMMetadata(
            context_window=8192,
            num_output=512,
            model_name=self.model,
            is_chat_model=True,
            is_function_calling_model=False
        )


In [84]:
from llama_index.core.evaluation import FaithfulnessEvaluator, RelevancyEvaluator

In [85]:
groq_llm = GroqLLM()
faith_eval = FaithfulnessEvaluator(llm=groq_llm)
relev_eval = RelevancyEvaluator(llm=groq_llm)

faith_result = faith_eval.evaluate_response(query=prompt,response=response,context=retrieved_context)

relev_result = relev_eval.evaluate_response(
    query=prompt,
    response=response
)

# Print results



## Semantic Search Metrics
### ROGUE BLEU BERTScore against GT answer


In [86]:
!pip install evaluate rouge_score
!pip install bert_score
import evaluate
from evaluate import load

rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")

generated = [str(response)]
reference = [gt_answer]

ROUGE1=rouge.compute(predictions=generated, references=reference)
BLEU1=bleu.compute(predictions=generated, references=reference)

bertscore = load("bertscore")
score = bertscore.compute(predictions=generated, references=reference, lang="en")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [88]:
faith_result

EvaluationResult(query='what is working capital? what does it tell you about a company?', contexts=['Working Capital = Current Assets – Current Liabilities \nIf the working capital is a positive number, it implies that the company has working \ncapital surplus and can easily manage its day to day operations. However if the \nworking capital is negative, it means the company has a working capital deficit. \nUsually if the company has a working capital deficit, they seek a working capital loan \nfrom their bankers. \nThe concept of ‘Working Capital Management’ in itself is a huge topic in Corporate \nFinance. It includes inventory management, cash management, debtor’s \nmanagement etc. The company’s CFO (Chief Financial Officer) strives to manage the \ncompany’s working capital efficiently. Of course, we will not get into this topic as we \nwill digress from our main topic. \nThe working capital turnover ratio is also referred to as Net sales to working capital. \nThe working capital tur

**Printing Results**

In [87]:
print("Results:\n")
print("Questioning LLM:", model_id)
print("Temperature:", llm.generate_kwargs.get("temperature"))
print("Judge LLM for Faithfulness & Relevance :",groq_llm.model)
print("Faithful:", faith_result.passing)
print("Relevant:", relev_result.passing)
print("\nBLEU + ROUGE Evaluation: \n ")
print("ROUGE rouge1:", ROUGE1['rouge1'])
print("ROUGE rouge2:", ROUGE1['rouge2'])
print("ROUGE rougeL:", ROUGE1['rougeL'])
print("ROUGE rougeLsum:", ROUGE1['rougeLsum'])
print("\nBLEU:", BLEU1['bleu'])
print("BLEU precisions:", BLEU1['precisions'])
print("BLEU brevity_penalty:", BLEU1['brevity_penalty'])
print("BLEU length_ratio:", BLEU1['length_ratio'])
print("BLEU translation length:", BLEU1['translation_length'])
print("BLEU reference length :", BLEU1['reference_length'])
print("\nBERTScore Evaluation:\n")
print(f"Precision: ",score['precision'])
print(f"Recall:",score['recall'])
print(f"F1 Score:",score['f1'])

Results:

Questioning LLM: Qwen/Qwen3-1.7B
Temperature: 0.0
Judge LLM for Faithfulness & Relevance : llama3-70b-8192
Faithful: True
Relevant: True

BLEU + ROUGE Evaluation: 
 
ROUGE rouge1: 0.38060606060606067
ROUGE rouge2: 0.1336573511543135
ROUGE rougeL: 0.19636363636363638
ROUGE rougeLsum: 0.36121212121212115

BLEU: 0.03990064601696472
BLEU precisions: [0.42298850574712643, 0.10829493087557604, 0.025404157043879907, 0.004629629629629629]
BLEU brevity_penalty: 0.8281957436793111
BLEU length_ratio: 0.8413926499032882
BLEU translation length: 435
BLEU reference length : 517

BERTScore Evaluation:

Precision:  [0.8273256421089172]
Recall: [0.8223039507865906]
F1 Score: [0.8248071670532227]


# Implement Rerank [working]
[Query Bundle](https://docs.llamaindex.ai/en/v0.10.17/api/llama_index.core.schema.QueryBundle.html#querybundle)

In [ ]:
#from llama_index.core.retrievers import VectorIndexRetriever (Already Imported)
from llama_index.core import QueryBundle
from llama_index.core.indices.postprocessor import LLMRerank

#from typing import List

retriever2 = VectorIndexRetriever(
        index=index,
        similarity_top_k=5,
        #vector_store_query_mode="default",

    )

query_bundle = QueryBundle(prompt)

retrieved_nodes = retriever2.retrieve(query_bundle)

reranker = LLMRerank(
            choice_batch_size=5,
            top_n=5,
        )
retrieved_nodes = reranker.postprocess_nodes(
            retrieved_nodes, query_bundle
        )

#pprint_response(retrieved_nodes,show_source=True)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:
query_engine2=RetrieverQueryEngine(retriever=retriever2)

In [ ]:
print(f"Retrieved {len(retrieved_nodes)} nodes:")
for i, node in enumerate(retrieved_nodes):
    print(f"\n--- Node {i + 1} ---")
    print(node.get_content())
    print(f"Score: {node.score}")

Retrieved 0 nodes:


In [ ]:
response2=query_engine2.query(query_bundle)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:
from llama_index.core.response.pprint_utils import pprint_response
pprint_response(response2,show_source=True)
#print(response)

Final Response: Working capital is the difference between current
assets and current liabilities. It indicates the amount of working
capital available to a company to meet its short-term obligations. If
the working capital is positive, it means the company has sufficient
working capital to meet its short-term obligations. If it is negative,
it means the company has a working capital deficit and may need to
seek a working capital loan from its bankers. ---------------------
Answer the question: what is working capital? what does it tell you
about a company?  The working capital is the difference between
current assets and current liabilities. It indicates the amount of
working capital available to a company to meet its short-term
obligations. If the working capital is positive, it means the company
has sufficient working capital to meet its short-term obligations. If
it is negative, it means the company has a working capital deficit and
may need to seek a working capital loan from its b

joining the nodes

In [ ]:
retrieved_context = " ".join([node.text for node in response2.source_nodes])

# POST RERANK EVALUATION

In [ ]:
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")

generated = [str(response2)]
reference = [gt_answer]

print("ROUGE:", rouge.compute(predictions=generated, references=reference))
print("BLEU:", bleu.compute(predictions=generated, references=[reference]))


##BERTScore
[Paper](https://arxiv.org/pdf/1904.09675)

In [ ]:
generated = [str(response2)]
reference = [retrieved_context]  # compare to retrieved content, not gold answer

P, R, F1 = score(generated, reference, lang="en")

print("\nBERTScore Evaluation:")
print(f"Precision: {P[0]:.4f}")
print(f"Recall:    {R[0]:.4f}")
print(f"F1 Score:  {F1[0]:.4f}")
